In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter
from subprocess import check_output
#看看文件是否存在，并且是不是用utf8
print(check_output(["ls", "input"]).decode("utf8"))

Authorization for Minor Form.rtf
child_wishlist_v2.csv
gift_goodkids_v2.csv
sample_submission_random_v2.csv



### 后面看懂代码后改成dataframe
```python
drop(0,1)的意思是：其中1表示axis=1,0表示名称，在这里即为删除名称为0的列
gift_pref_df = pd.read_csv('../input/child_wishlist_v2.csv',header=None).drop(0,1)
child_pref_df = pd.read_csv('../input/gift_goodkids_v2.csv',header=None).drop(0,1)
```

In [3]:
#居然转换成了ndarry
#孩子喜欢的礼物
gift_pref = pd.read_csv('input/child_wishlist_v2.csv',header=None).drop(0, 1).values
#礼物喜欢的孩子
child_pref = pd.read_csv('input/gift_goodkids_v2.csv',header=None).drop(0, 1).values

In [4]:
type(child_pref)

numpy.ndarray

In [6]:
print(np.min(gift_pref))
print(np.max(gift_pref))

0
999


In [7]:
print(np.min(child_pref))
print(np.max(child_pref))

0
999999


In [5]:
#有1000000个children
n_children = 1000000 
#1000种gift
n_gift_type = 1000
#每种gift1000个
n_gift_quantity = 1000 
#每个孩子有喜欢的100个礼物。(孩子开心)
n_gift_pref = 100 # number of gifts a child ranks
#每个礼物优先考虑1000个好孩子（圣诞老人开心）
n_child_pref = 1000 # number of children a gift ranks
#0-5000为三胞胎，triplets=50001 0.5%
triplets = math.ceil(0.005 * n_children / 3.) * 3    
#5001-45000双胞胎，即45000-5001+1=40000
twins = math.ceil(0.04 * n_children / 2.) * 2    
#根据目标函数里面定义的
ratio_gift_happiness = 2
ratio_child_happiness = 2

### 1.构造child偏好列表和gift编号列表

In [8]:
#非双胞胎和非三胞胎的孩子，因为成pair，因此-6只是占位符
class Child(object):
    
    def __init__(self, idx, prefer):
        # 每个孩子有自己的ID和prefer列表，这里使用字典来存储prefer列表中的giftID和对应的happiness，
        self.idx = idx
        self.prefer_dict = dict()
        # 这里的happiness使用包含两个值的列表存储，前者是礼物为此giftID的happiness，后者是礼物不在prefer列表的happiness
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = [12*(prefer.shape[0] - i), -6]
    
    def add_gifts_prefer(self, giftid, score):
        # 添加gift对child的prefer，前者为孩子对礼物的happiness，后者对礼物对孩子的happiness
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 6*score
        else:
            self.prefer_dict[giftid] = [-6, 6*score] 
        return None
        
    def happiness(self, giftid):
        # 获取当前childID和指定giftID的happiness对
        # 若giftID不在prefer字典中，则返回[-6,-6]
        return self.prefer_dict.get(giftid, [-6, -6])

In [9]:
#双胞胎
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        # 构造twins的prefer字典，用giftID在两个prefer列表中的score之和的三倍（平均score的6倍）和-6作为value
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [3*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 3*score
        else:
            self.prefer_dict[giftid] = [-6, 3*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])

In [10]:
#三胞胎
class Child_triplet(object):
"""对于三胞胎，三个人要作为整体考虑
param idx:child id 
param prefer1:三胞胎中第一个孩子偏爱的礼物
param prefer2:三胞胎中第二个孩子偏爱的礼物
param prefer3:三胞胎中第三个孩子偏爱的礼物
"""
    def __init__(self, idx, prefer1, prefer2, prefer3):
        
        self.idx = idx
        self.prefer_dict = dict()
        #三胞胎三个人，有三组不同的喜欢的礼物，所以对于三胞胎set(list(prefer1) + list(prefer2) + list(prefer3))肯定大于100
        for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):
            score = 0
            #对于三胞胎中的第一个人，偏爱有各种礼物的得分
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            #对于三胞胎中的第二个人
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            #对于三胞胎中的第三个人
            if p in list(prefer3):
                score += 2*(100 - list(prefer3).index(p))
            else:
                score -= 1
            #这个是在干啥？
            self.prefer_dict[p] = [2*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        #最佳匹配
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 2*score
        else:
            self.prefer_dict[giftid] = [-6, 2*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])

#### 得到一个孩子喜欢的礼物
gift_pref[index]
#### 三胞胎孩子的礼物
Child_triplet(0, gift_pref[0], gift_pref[1], gift_pref[2])
#### 双胞胎孩子的礼物
Child_twin(5001, gift_pref[5001], gift_pref[5002])
#### 普通孩子
Child(45001, gift_pref[45001])


### 小例子解释作者是在干啥
```python
C=[]
C.append(Child_triplet(0, gift_pref[0], gift_pref[1], gift_pref[2]))
C.append(Child_twin(5001, gift_pref[5001], gift_pref[5002]))
C.append(Child(45001, gift_pref[45001]))

#由于index=0的孩子是三胞胎，因此它偏爱的礼物数是大于100的,从下面的几个条语句可以看出，0号孩子也有偏爱0号礼物的。
#虽然它不在0号孩子偏爱的礼物中，但它在1号孩子偏爱的礼物中。
print(np.where(gift_pref[0]==0))
print(np.where(gift_pref[1]==0))
print(gift_pref[1][40])
print(np.where(gift_pref[2]==0))
(array([], dtype=int64),)
(array([40]),)
0
(array([], dtype=int64),)
#第0个孩子匹配第0个礼物的得分（这样存储的有个问题如果没有礼物则会直接报错）
C[0].prefer_dict[0]
[236, -6]
```

In [33]:
# 构造孩子的偏好列表及其匹配礼物的happiness
Children = []
for i in range(0, 5001, 3):
    Children.append(Child_triplet(i, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+1, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+2, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
for i in range(5001, 45001, 2):
    Children.append(Child_twin(i, gift_pref[i], gift_pref[i+1]))
    Children.append(Child_twin(i+1, gift_pref[i], gift_pref[i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(45001, 1000000)]

In [38]:
for j in range(1000):# 遍历所有的giftID
    #每类礼物更偏爱的孩子
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):# 遍历一个giftID下所有的preferred kid ID
        #同一礼物如果是三胞胎则以三胞胎中最先出现的为主。
        # 如果是三胞胎中的一个，而且没有被处理，就一次性处理三胞胎
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            Children[cid3].add_gifts_prefer(j, score_)
        #如果是双胞胎，以双胞胎最先出现的为主
        # 如果是twins中的childID，而且没有被处理，就一次性处理twins
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 1:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            #如果是普通孩子
        elif cf[i] > 45000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

### 2.构建最小花费流模型

- 这里将孩子和礼物的一一配对问题抽象为将1,000,000个礼物分发给1,000,000个孩子的最小花费流问题。
- 我们需要将礼物分发为孩子，每个礼物和孩子之间理论上都有一条通路，通路的容量为1，表示一个礼物只能分发给一个孩子；同时每条通路都有一个cost，我们需要找到cost最小的分发方式。
- 通路的cost需要从这条通路连接的孩子和礼物的happiness来计算得到。
- 此外还有一些限制条件，三胞胎、双胞胎需要一样的礼物；每种礼物只有一千个。
- 三胞胎和双胞胎需要一样的礼物：通过定义从礼物到孩子的分发过程实现，在礼物分发的过程中，将三胞胎、双胞胎视为一个节点，即只给一个节点分发礼物，容量设为3、2，cost如何计算？
- 每种礼物的数量可以通过设定节点的供应量supply来限制，将孩子节点的supply设为-1，表示需要1个礼物；将每种礼物结点的supply设为1000，表示只有1000个礼物，保证了整个网络的supply之和为0；此外还可以设立一些中转结点，supply=0.

In [ ]:
from ortools.graph import pywrapgraph
# 最终happiness计算中的权重
W_CHILD = 9020
W_GIFTS = 2

In [41]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()
# 这里将孩子和礼物的
# 开始结点，这里为礼物ID，取值为1000000-1000999
start_nodes = []
# 结束结点，这里为孩子ID，取值为0-999999
end_nodes = []
# 两个节点之间线路的容量，这里为从礼物结点到孩子结点可分发的礼物数量，由于三胞胎和双胞胎分发礼物时只给第一个孩子分发，
# 所以三胞胎可分发礼物为3，双胞胎为2，其他人为1
capacities = []
# 单位花费，指从开始结点到结束结点分发一次礼物的cost，由于是最小花费流问题，
# 这里为一次礼物分发过程（即礼物和孩子配对过程）的孩子happiness和礼物happiness的加权求和的负值
unit_costs = []
# 注意：这里的开始结点、结束结点、容量、单位花费列表的元素是一一对应的，

In [ ]:
# 1.定义网络结构（包括开始结点、结束结点、边容量、单位花费）

# triplets
# 对于三胞胎的第一个孩子，对于其喜好列表中的每一个礼物，添加从礼物到孩子的网络通路（包括容量3和花费）
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        # 权重为什么是10000和0，为什么要加6？为了防止有负数？
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# twins
# 对于双胞胎的第一个孩子，对于其喜好列表中的每一个礼物，添加从礼物到孩子的网络通路（包括容量2和花费）
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
# 对于其他的每一个孩子，对于其喜好列表，添加从喜好列表中每一个礼物到此孩子的通路（包括容量1和花费）
for i in range(45001, 1000000):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

In [ ]:
# 2.add Arc；在节点之间添加边

# gift -> children；
# 添加从礼物到孩子的分发花费
for i in range(len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
# 1001000结点为孩子节点的集合结点，添加从每个收到礼物的孩子结点到此节点的一条路径，容量为收到礼物的数量，花费为0
for i in range(0, 5001, 3):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
# 1001001结点为礼物结点的汇总节点，添加从每个礼物结点到汇总节点的容量为1000花费为0的通路
for i in range(1000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
# 添加从开始结点的汇总节点到结束结点的汇总节点的容量为1000000花费为0的通路
min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


In [ ]:
# 3.add Supply：设置结点supply供应量，结点供给量之和为0

# 将每个礼物结点的供应量设为1000，表示供给结点
for i in range(1000):
    min_cost_flow_1.SetNodeSupply(int(1000000+i), int(1000))

# 设置孩子节点的供应量
# 将单个孩子结点的供应量设置为0，相当于中转节点
for i in range(0, 5001, 3):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
# 将礼物结点汇总节点的供应量设置为0，为中转结点
min_cost_flow_1.SetNodeSupply(int(1001001), int(0)) 
# 将孩子汇总节点的供应量设置为-1000000，表示需要1000000份礼物，为接收节点
min_cost_flow_1.SetNodeSupply(int(1001000), int(-1000000)) 

In [ ]:
min_cost_flow.Solve()

In [ ]:
assignment = [-1]*1000000
twins_differ = []
triplets_differ = []

#for i in well_assigned:
#    assignment[i] = assignment_0[i]

for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) != 0 and min_cost_flow.Head(i) < 1000000:
        c = min_cost_flow.Head(i)
        g = min_cost_flow.Tail(i)
        f = min_cost_flow.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))


### 3.计算平均归一化幸福感

#### 求最大公约数math.gcd
```python
greatest common divisor of x and y
math.gcd(319,377)
29
```
- 最大公约数=A×B/最小公倍数,下面的函数实际上是求的最小公倍数
```python
def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)
```


In [35]:
def lcm(a, b):
    """Compute the lowest common multiple of a and b，求的是最小公倍数"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)

In [ ]:
# kaggle给的计算平均归一化幸福感的函数
def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if triplets have the same gift
    for t1 in np.arange(0,triplets,3):
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1[1] == triplet1[1] and triplet2[1] == triplet3[1]
                
    # check if twins have the same gift
    for t1 in np.arange(triplets,triplets+twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        # print(t1)
        assert twin1[1] == twin2[1]

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))

    # to avoid float rounding error
    # find common denominator
    # NOTE: I used this code to experiment different parameters, so it was necessary to get the multiplier
    # Note: You should hard-code the multipler to speed up, now that the parameters are finalized
    denominator1 = n_children*max_child_happiness
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type
    common_denom = lcm(denominator1, denominator2)
    multiplier = common_denom / denominator1

    # # usually denom1 > demon2
    return float(math.pow(total_child_happiness*multiplier,3) + math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    # return math.pow(float(total_child_happiness)/(float(n_children)*float(max_child_happiness)),2) + math.pow(np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity),2)

In [ ]:
# 计算给出的随机提交结果的幸福感
random_sub = pd.read_csv('../input/sample_submission_random_v2.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

- output:normalized child happiness= 0.04621203 , normalized gift happiness -4.59355e-05 9.868817990273061e-05